# Qwen2.5-Math-7B HuggingFace Inference Pipeline

> **Purpose**: Generate responses from Qwen2.5-Math-7B-Instruct model on 30 linear algebra problems

**Notebook**: `4_qwen_math_7b_hf_inference_and_verification_pipeline.ipynb`  
**Author**: Dr. Shradha's Research Team | **Date**: 2025-12-11 | **Version**: 1.0.0

## Step 1: Install Dependencies

Install required packages for HuggingFace inference.

In [ ]:
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import re
import time

# ================================================================================
# CONFIGURATION & FILE DEFINITIONS
# ================================================================================

INPUT_FILE = "data/ds2_qna.xlsx"
OUTPUT_FILE = "data/qna_responses_qwen2.5-Math-7b.xlsx"
CHECKPOINT_PREFIX = "qwen_math_7b_checkpoint_row"

print("Configuration loaded:")
print(f"  Input:  {INPUT_FILE}")
print(f"  Output: {OUTPUT_FILE}")

Configuration loaded:
  Input:  data/ds2_qna.xlsx
  Output: data/qna_responses_qwen2.5-Math-7b.xlsx


## Step 2: Mount Google Drive & Set Working Directory

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/MS&T_Projects/LLM_Tasks/Visual QnA')

In [ ]:
# Load the main dataset
df = pd.read_excel(INPUT_FILE)
print(f"Loaded {len(df)} problems")
print("Columns:", list(df.columns))
print('\nSample of id and instruction columns:')
print(df[['id', 'instruction']].head())

Loaded 30 problems
Columns: ['id', 'level', 'category', 'problem_text', 'problem_latex', 'answer_latex', 'instruction']

Sample of id and instruction columns:
                id                                        instruction
0  L3_det_3x3_7420  Solve this linear algebra problem. Show your w...
1  L3_det_3x3_8513  Solve this linear algebra problem. Show your w...
2  L3_det_3x3_1161  Solve this linear algebra problem. Show your w...
3  L3_det_3x3_9684  Solve this linear algebra problem. Show your w...
4     L3_mult_9996  Solve this linear algebra problem. Show your w...


In [ ]:
# Load the model and tokenizer
model_name = "Qwen/Qwen2.5-Math-7B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Loading model: {model_name}")
print(f"Device: {device}")

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16 if device == "cuda" else torch.float32,
    device_map="auto",
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

print("Model and tokenizer loaded successfully!")

Loading model: Qwen/Qwen2.5-Math-7B-Instruct
Device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/658 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Model and tokenizer loaded successfully!


In [ ]:
def get_qwen_math_response(instruction, model, tokenizer, device):
    """
    Get response from Qwen Math model
    """
    try:
        # Format with Chain of Thought prompting
        messages = [
            {"role": "user", "content": instruction}
        ]

        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        model_inputs = tokenizer([text], return_tensors="pt").to(device)

        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=4000,
            temperature=0,
            pad_token_id=tokenizer.eos_token_id
        )

        # Remove input tokens from generated output
        generated_ids_trimmed = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        response = tokenizer.batch_decode(generated_ids_trimmed, skip_special_tokens=True)[0]

        return response
    except Exception as e:
        return f"Error: {str(e)}"

# Test with an example
example_instruction = df.iloc[0]['instruction']
print("Example instruction:", example_instruction[:200], "...")

# Uncomment to test with example
# example_response = get_qwen_math_response(example_instruction, model, tokenizer, device)
# print("Example response:", example_response[:500], "...")

Example instruction: Solve this linear algebra problem. Show your work and give the final answer.

Find the determinant of the 3×3 matrix A.

A = \begin{bmatrix} 18.25 & 8.75 & -5.50 \\ -12.75 & 0.75 & 18.20 \\ -1.50 & 2. ...


In [10]:
# Process each row and get responses for the Qwen-Math_7B column where it's empty
print(f"Starting to process {len(df)} rows...")

for idx, row in df.tail(10).iterrows():
    # Check current value in Qwen-Math_7B column
    current_response = row.get('Qwen-Math_7B', pd.NA)

    # Only process if the response is empty/NaN
    if pd.isna(current_response) or current_response == "" or str(current_response) == "nan":
        print(f"Processing row {idx+1}/{len(df)}: {row['id']}")

        instruction = row['instruction']
        response = get_qwen_math_response(instruction, model, tokenizer, device)

        # Add response to dataframe
        df.at[idx, 'Qwen-Math_7B'] = response

        print(f"  Got response ({len(response)} chars): {response[:100]}...")

        # Save periodically to prevent data loss
        if (idx + 1) % 5 == 0:  # Save every 5 rows
            df.to_excel(f'{CHECKPOINT_PREFIX}_{idx+1}.xlsx', index=False)
            print(f"  Saved checkpoint at row {idx+1}")

        # Add a small delay to be respectful to resources
        time.sleep(0.5)
    else:
        print(f"Skipping row {idx+1}/{len(df)}: {row['id']} (already has response)")

print("Completed processing all rows!")

Starting to process 30 rows...
Processing row 21/30: L5_sym_square_9808
  Got response (1492 chars): To compute \( A^2 \) for the general matrix \( A = \begin{bmatrix} a & b \\ c & d \end{bmatrix} \), ...
Processing row 22/30: L5_sym_square_4267
  Got response (1492 chars): To compute \( A^2 \) for the general matrix \( A = \begin{bmatrix} a & b \\ c & d \end{bmatrix} \), ...
Processing row 23/30: L5_sym_square_8668
  Got response (1399 chars): To compute \( A^2 \) for the lower triangular matrix \( A \), we start by multiplying \( A \) by its...
Processing row 24/30: L5_sym_square_1412
  Got response (1399 chars): To compute \( A^2 \) for the lower triangular matrix \( A \), we start by multiplying \( A \) by its...
Processing row 25/30: L5_sym_det_8587
  Got response (587 chars): To find the determinant of the matrix \( A = \begin{bmatrix} a & b \\ 0 & c \end{bmatrix} \), we use...
  Saved checkpoint at row 25
Processing row 26/30: L5_sym_det_8164
  Got response (524 chars): To find 

In [11]:
# Save the final dataframe with Qwen-Math responses
df.to_excel(OUTPUT_FILE, index=False)
print(f"Final dataset saved to {OUTPUT_FILE}!")

Final dataset saved to data/qna_responses_qwen2.5-Math-7b.xlsx!


In [12]:
# Verify the results
print('Final results summary:')
for idx, row in df.iterrows():
    qwen_resp = row.get('Qwen-Math_7B', pd.NA)
    if not pd.isna(qwen_resp) and qwen_resp != "" and str(qwen_resp) != "nan":
        print(f"{row['id']}: Response length = {len(str(qwen_resp))}")
    else:
        print(f"{row['id']}: NO RESPONSE")

Final results summary:
L3_det_3x3_7420: NO RESPONSE
L3_det_3x3_8513: NO RESPONSE
L3_det_3x3_1161: NO RESPONSE
L3_det_3x3_9684: NO RESPONSE
L3_mult_9996: NO RESPONSE
L3_mult_7019: NO RESPONSE
L3_mult_8392: NO RESPONSE
L3_inv_2495: NO RESPONSE
L3_inv_1854: NO RESPONSE
L3_inv_8125: NO RESPONSE
L4_det_5x5_7616: NO RESPONSE
L4_det_5x5_2648: NO RESPONSE
L4_det_5x5_5496: NO RESPONSE
L4_det_6x6_2184: NO RESPONSE
L4_det_6x6_3368: NO RESPONSE
L4_det_6x6_5752: NO RESPONSE
L4_eig_4x4_2081: NO RESPONSE
L4_eig_4x4_9130: NO RESPONSE
L4_rank_5x5_5611: NO RESPONSE
L4_rank_5x5_1417: NO RESPONSE
L5_sym_square_9808: Response length = 1492
L5_sym_square_4267: Response length = 1492
L5_sym_square_8668: Response length = 1399
L5_sym_square_1412: Response length = 1399
L5_sym_det_8587: Response length = 587
L5_sym_det_8164: Response length = 524
L5_sym_det_9867: Response length = 524
L5_sym_inv_6988: Response length = 1221
L5_sym_inv_3967: Response length = 2043
L5_sym_inv_2122: Response length = 1318
